In [ ]:
import os
import sys
import fnmatch
import numpy as np
import pandas as pd
import json
import gzip
import pickle
import csv
import scipy.sparse
Xauth = None
from collections import defaultdict

In [ ]:
with gzip.open('useful_venue_list.pkl.gz','rb') as fp:
    all_venues = pickle.load(fp)
with gzip.open('useful_authors_list.pkl.gz','rb') as fp:
    all_authors = pickle.load(fp)
with gzip.open('useful_papers.pkl.gz','rb') as fp:
    all_papers = pickle.load(fp)

In [ ]:
conf_idx = {v:i for i,v in enumerate(all_venues)}
name_idx = {v:i for i,v in enumerate(all_authors)}
n_confs = len(all_venues)
n_auths = len(all_authors)
r1_confs = pickle.load(open('old_version/r1_confs.pkl','rb'))
r1_confs_dict = {_:1 for _ in r1_confs}

In [ ]:
faculty_affil = pd.read_csv('faculty-affiliations.csv')
ranks = pd.read_csv('ranks.csv')
def csv2dict_str_str(fname):
    with open(fname, mode='r') as infile:
        rdr = csv.reader(infile)
        d = {rows[0].strip(): rows[1].strip() for rows in rdr}
    return d
aliasdict = csv2dict_str_str('dblp-aliases.csv')

In [ ]:
min_year = all_papers[0][6]
max_year = all_papers[-1][6]
span_years = max_year - min_year + 1

In [ ]:
auth_years = np.ones((n_auths,2)) * np.array([3000,1000]) 
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    for a in authors:
        i = name_idx[a]
        auth_years[i,0] = min(auth_years[i,0],year)
        auth_years[i,1] = max(auth_years[i,1],year)

In [ ]:
clf_files = ['weights_faculty_above6_log_1_40_25_0.pkl',
             'weights_salary_above6_linear_1_0_25_0.pkl',
              'weights_nsfmarginal_above6_log_1_0_25_0.pkl']
clf_files2 = ['weights_salary_above6_linear_1_0_25_1.pkl',
            'weights_nsfmarginal_above6_log_1_0_25_1.pkl',
            'weights_faculty_above6_log_1_40_25_7.pkl']
clf_files = ['weights_salary_above6_linear_50_0_25_0.pkl',
            'weights_nsfmarginal_above6_log_50_0_25_0.pkl',
            'weights_faculty_above6_log_50_40_25_0.pkl']

clfs = [pickle.load(open(c,'rb')) for c in clf_files]
clfs2 = [pickle.load(open(c,'rb')) for c in clf_files2]

span_years = 50 
repeat_needed = span_years//(clfs[0].shape[0]//n_confs)
if repeat_needed > 1:
    clfs = [np.repeat(_,repeat_needed) for _ in clfs]
clfs = clfs2
clfs = [np.squeeze(_) for _ in clfs]

print([_.shape for _ in clfs])
clp =  18 #7
clfs2 = []
if True:
    for result_clf in clfs:
        result_clf = result_clf.reshape((-1,span_years))

        #plt.plot(result_clf.sum(0)/result_clf.sum(0).sum(),label='sum')
        print(abs(result_clf.mean(0)).mean(),abs(result_clf.std(0)).mean())
        #result_clf = np.minimum(30,np.maximum(result_clf,-30))
        #result_clf = (result_clf)/result_clf.std(0)
        result_clf = (result_clf-result_clf.mean(0))/result_clf.std(0)
        result_clf = result_clf.reshape((-1))
        clfs2.append(result_clf)
else:
    clfs2 = [(c-c.mean(0))/c.std(0) for c in clfs]
clfs = clfs2
clfs = [np.minimum(clp,np.maximum(-clp,c)) for c in clfs]
clfs = np.vstack(clfs)
clf = np.mean(clfs,0)


clf_gold = np.copy(clf)
#clf_gold = np.ones_like(clf)
#clf = clf_gold

In [ ]:
conf_ord = np.argsort(np.squeeze(clf))
#'Scientometrics','JCDL','NIPS',
conf_choice = ['ICRA','ICML','SIGGRAPH','HRI','ECCV','Comput. Graph. Forum','Shape Modeling International','Symposium on Geometry Processing','Computer Aided Geometric Design','I. J. Robotics Res.','CVPR','International Journal of Computer Vision','Robotics: Science and Systems','ICRA','WACV','ICML','AISTATS','CoRR','SIGGRAPH Asia','ECCV','ICCV','ISER','Humanoids','3DV','IROS','CoRL','Canadian Conference on AI','ACCV','Graphics Interface','CRV','BMVC']
ri_confs = np.zeros(clf_gold.shape[0])
ms = clf_gold.mean()
ss = clf_gold.std()
np.set_printoptions(precision=1)
seen = {}
for i in range(clf_gold.shape[0]):
    idx = conf_ord[-(i+1)]
    conf_name = all_venues[idx//span_years]
    conf_score = clf[idx]
    if conf_name in conf_choice:
        ri_confs[idx] = 1
    if conf_name in conf_choice and conf_name not in seen:
        vec = clfs[:,idx]
        print('{:20s}{}\t{:.1f}\t{}'.format(conf_name[:20],str(min_year + (idx % span_years)),(conf_score-ms)/ss,vec))
        seen[conf_name] =1
ri_confs.shape,ri_confs.sum()

In [ ]:
top_k = 50
i = -1
j = 0
seen = {}
while j < top_k:
    i += 1
    idx = conf_ord[-(i+1)]
    conf_name = all_venues[idx//span_years]
    if conf_name in seen:
        continue
    j+=1
    conf_score = clf[idx]
    seen[conf_name] = 1
    print('{:20s}\t{}\t\t{:.3f}\t{:.2f}'.format(conf_name[:18],min_year + (idx % span_years),100*conf_score,(conf_score-ms)/ss))

In [ ]:
import matplotlib.pyplot as plt
for clf in [clf_gold]:
    plt.style.use('fivethirtyeight')

    plt.style.use('seaborn-whitegrid')

    conf_choice2 = ['SIGGRAPH','AAAI','NIPS','CVPR','ICRA','ICML','ICCV','ECCV',
                   'International Journal of Computer Vision','Robotics: Science and Systems']
    #conf_choice2 = ['CVPR','ECCV','ICCV','International Journal of Computer Vision','Robotics: Science and Systems']
    #conf_choice2 = ['SIGMOD Conference','VLDB','ICDT','KDD','ACM Trans. Inf. Syst.','PODS']
    #conf_choice2 = ['ACL','EMNLP','EACL']

    conf_choices = [conf_choice2, 
                    ['STOC','FOCS','SODA','EC','WINE'],
                    ['UAI','AAAI','IJCAI','ICML','NIPS'],
                    ['ECCV','ICCV','CVPR','BMVC','International Journal of Computer Vision','IEEE Trans. Pattern Anal. Mach. Intell.'],
                    ['Robotics: Science and Systems','IROS','ICRA','WAFR','ISER'],
                    ['SIGGRAPH','SIGGRAPH Asia','ACM Trans. Graph.','Graphics Interface'],
                    ['SIGIR','JCDL','CIKM','KDD','WWW','SIGMOD Conference']
                   ]
    labels = ['All','Theory','AI','Vision','Robotics','Graphics','IR']
    #conf_choices = [['Robotics: Science and Systems','IROS','ICRA','CoRL','WAFR','HRI','ISER']]
    for conf_choice2,l in zip(conf_choices,labels):
        plt.figure()
        #conf_choice2 = 
        conf_choice3 = []
        vs = clf.std()
        for conf in conf_choice2:
            idx = conf_idx[conf]
            s = clf[span_years*idx+1999-1970]#max(clf[span_years*idx:span_years*(idx+1)]) ##s = clf[span_years*idx+2015-1970]#
            conf_choice3.append((s,conf))
        plt.figure(figsize=(12,8))
        for s,conf in sorted(conf_choice3,reverse=True):
            idx = conf_idx[conf]
            _ = plt.plot(np.arange(min_year,max_year+1)[:-2],(clf[span_years*idx:span_years*(idx+1)]/vs)[:-2],label=conf,lw=5)
        plt.grid(True)
        plt.xlabel('year')
        plt.ylabel('value')
        #plt.title('WITH Temporal Adjustment')
        plt.title(l)
        plt.legend()
        #plt.ylim(0,25)
        plt.xlim(1970,2020)
        plt.xticks(np.arange(1970,2020,10),[str(_) for _ in np.arange(1970,2020,10)])
        plt.tight_layout()
        plt.savefig(l + '_t.pdf',facecolor='w',edgecolor='w')
        
        #plt.show()

    #plt.show()
clf = clf_gold
plt.style.available

In [ ]:
for style in []:##plt.style.available:#:
    plt.style.use(style)
    plt.figure(figsize=(12,8))
    for s,conf in sorted(conf_choice3,reverse=True):
        idx = conf_idx[conf]
        weights = [result_clf[years_per_conf*idx + yr]/vs for yr in offset_years]
        _ = plt.plot(np.arange(min_year,max_year+1),weights,label=conf,lw=5)
    plt.grid(True)
    plt.xlabel('year')
    plt.ylabel('value')
    plt.title(style)
    plt.legend()

In [ ]:
if Xauth is None:
    count_vecs = {}
    paper_vecs = []
    for paper in all_papers:
        tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
        n = len(authors)
        j = 50*conf_idx[venue] + (year-min_year)

        if n not in count_vecs:
            author_scores = np.ones(n) #1/(np.arange(n)+1) 
            #author_scores[-1] = author_scores[0]
            author_score_sum = author_scores.sum()
            #author_scores /= author_score_sum
            count_vecs[n] = author_scores #/ author_score_sum
        else:
            author_scores = count_vecs[n]
            paper_vecs.append([(name_idx[a],j,v) for a,v in zip(authors,author_scores)])

In [ ]:
if Xauth is None or Xauth.shape[1] != 50*n_confs:
    import scipy.sparse
    Xauth = scipy.sparse.dok_matrix((n_auths,50*n_confs))
    xdict = {}
    if False:
        for paper in all_papers:
            tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
            n = len(authors)
            j = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS
            for ai in range(n):#zip(count_vecs[n],authors):
                i = name_idx[authors[ai]]
                #xdict[(i,j)] = 1/n + xdict.get((i,j),0)
                xdict[(i,j)] = count_vecs[n][ai] + xdict.get((i,j),0)

    else:
        for paper_vec in paper_vecs:
            for i,j,v in paper_vec:
                xdict[(i,j)] = v + xdict.get((i,j),0)

    Xauth._update(xdict)
            
    Xauth = scipy.sparse.csr_matrix(Xauth)

In [ ]:
paper_counts = np.zeros(n_auths)
paper_counts_c = np.zeros(n_confs)
paper_counts_year = {}#np.zeros(n_auths)
paper_counts_year_counter = np.zeros((max_year-min_year+1))

for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    paper_counts_c[conf_idx[venue]] += 1
    paper_counts_year_counter[year-min_year] += 1
    for a in authors:
        i = name_idx[a]
        paper_counts[i] += 1
        paper_counts_year[(a,year)] = 1+ paper_counts_year.get((a,year),0)

In [ ]:
total_scores = Xauth.dot(clf_gold)

In [ ]:
page_rank_people = pickle.load(open('new_pagerank_people.pkl','rb'))
page_rank_confs = pickle.load(open('new_pagerank_conf.pkl','rb'))
page_rank_clf = np.repeat(page_rank_confs,50)
page_rank_scores = Xauth.dot(page_rank_clf)


In [ ]:
df_corr = pd.read_csv('correlation_cleaned.csv')
df_corr = df_corr.drop(columns=[_ for _ in df_corr.columns if 'Unnamed' in _])
df_corr = df_corr.drop(columns=['pms','n_papers'])
df_corr = df_corr.rename(columns={'totals': 'venue_score', 'csrp': 'csr_pubs','csrpn': 'csr_adj','gcite': 'influence'})
df_corr = df_corr[['name','papers', 'citations', 'h-index',
       'i10','csr_pubs', 'csr_adj','venue_score','influence']]
df_corr = df_corr.dropna('index')
df_corr.index = df_corr.name
df_corr['pagerankA'] = df_corr.venue_score
df_corr['pagerankC'] = df_corr.venue_score

ts = total_scores.std()
tm = total_scores.mean()
for name in df_corr.name:
    if name in name_idx:
        idx = name_idx[name]
        df_corr.loc[name,'venue_score'] = total_scores[idx]
        df_corr.loc[name,'pagerankA'] = page_rank_people[idx]
        df_corr.loc[name,'pagerankC'] = page_rank_scores[idx]
        #print(name,df_corr.loc[name,'papers'],paper_counts[idx])
        df_corr.loc[name,'papers'] = paper_counts[idx]


print(df_corr.corr('spearman').loc['influence','venue_score'],df_corr.corr('kendall').loc['influence','venue_score'],df_corr.corr('spearman').loc['h-index','venue_score'])
#if clfn == clfs_test.shape[-1]:
df_corr[['papers','citations','h-index','i10','csr_pubs','venue_score','pagerankA','pagerankC','influence']].corr('spearman')

In [ ]:
df_merged = pd.read_csv('faculty_affil_scholar.csv')
fuzzythresh=0.9 #0.9
df_merged.loc[df_merged.fuzzyscore > fuzzythresh,'dblpname'] = df_merged[df_merged.fuzzyscore > fuzzythresh].fuzzyname
df_merged.loc[df_merged.fuzzyscore > fuzzythresh,'dblpexists'] = 1
df_merged = df_merged[df_merged.dblpexists == 1]
df_merged['venue_score'] = np.ones_like(df_merged.dblpexists)
df_merged['pagerankA'] = np.ones_like(df_merged.dblpexists)
df_merged['pagerankC'] = np.ones_like(df_merged.dblpexists)
df_merged['papers'] = np.ones_like(df_merged.dblpexists)

ts = total_scores.std()
tm = total_scores.mean()
df_merged = df_merged.set_index(df_merged.dblpname)
seen_map = {}
for name in df_merged.index:
    if name in name_idx:
        idx = name_idx[name]
        seen_map[name] = 1
        df_merged.loc[name,'venue_score'] = total_scores[idx]
        df_merged.loc[name,'pagerankA'] = page_rank_people[idx]
        df_merged.loc[name,'pagerankC'] = page_rank_scores[idx]
        df_merged.loc[name,'papers'] = paper_counts[idx]
df_merged = df_merged.drop(columns=['Unnamed: 0','First Name','Last Name','Sholar link','Rank (Full, Associate, Assistant, Other)','Full Name','University_y','University_x','Unnamed: 11',"ID",'fuzzyname','dblpexists','fuzzyscore','UniversityID'])

In [ ]:
new_datas = []
for row in faculty_affil.itertuples():
    if row[1] in name_idx and row[1] not in seen_map:
        idx = name_idx[row[1]]
        seen_map[row[1]] = 1
        new_data = {}
        new_data['dblpname'] = row[1]
        #new_data['index'] = row[1]
        new_data['school'] = row[2]
        new_data['venue_score'] = total_scores[idx]
        new_data['pagerankA'] = page_rank_people[idx]
        new_data['pagerankC'] = page_rank_scores[idx]
        new_data['papers'] = paper_counts[idx]
        
        new_datas.append(new_data)
        #df_merged = df_merged.append([row[1],np.nan,np.nan,np.nan,row[2],row[1],(total_scores[idx]-tm)/ts])
df_csr_to_add = pd.DataFrame(new_datas)
#df_csr_to_add = df_csr_to_add.set_index('dblpname')
df_csr_to_add = df_csr_to_add.set_index('dblpname')
#df_merged = pd.concat([df_merged,df_csr_to_add])
print(df_merged.shape)
#print(faculty_affil.shape)
df_t10 = df_merged[df_merged['t10-index'].notna()]


In [ ]:
school_vals = df_t10.groupby('school').aggregate('sum').sort_values('venue_score',0,False)
t10_schools = school_vals.sort_values('t10-index',0,False)
school_vals.corr('spearman')

In [ ]:
df_msar = pd.read_csv('traditional_conf_scores.csv')
offset_years = [i for i in range(span_years)]
years_per_conf = 50

In [ ]:
df_msar_filt = df_msar[df_msar.dblp_name != 'NotAConf'].copy()#.sort_values('H',ascending=False)
scores = []
paperss = []
pagerankc = []
for row in df_msar_filt.itertuples():
    conf = row[-1]
    idx = conf_idx[conf]
    weights = [result_clf[years_per_conf*idx + yr] for yr in offset_years[1984-min_year:2014-min_year]]
    #scores.append(result_clf[years_per_conf*idx + offset_years[2014-min_year]])

    scores.append(np.max(np.array([w for w in weights])))
    pagerankc.append(page_rank_confs[idx])
    paperss.append(paper_counts_c[idx])

    #print(scores[-1],weights)

In [ ]:
df_msar_filt['venue_scores'] = scores
df_msar_filt['pagerankc'] = pagerankc
df_msar_filt['papers'] = paperss

df_msar_filt = df_msar_filt[~df_msar_filt.dblp_name.duplicated()].copy()
df_msar_filt = df_msar_filt[~df_msar_filt.venue_scores.duplicated()].copy()
#df_msar_filt['h-approx'] = 0.54*np.sqrt(df_msar_filt.citations)

df_msar_filt.shape

In [ ]:
thresh = 0
df_msar_filt = df_msar_filt[['papers','citations','H','pagerankc','venue_scores']]
print(df_msar_filt[df_msar_filt.H > thresh].shape)
df_msar_filt[df_msar_filt.H > thresh].corr('spearman')


In [ ]:
bp2 = pd.read_csv('uni_rank_bp.csv')
times = pd.read_csv('uni_rank_times.csv')

srf2 = pd.read_csv('uni_rank_mergedscholar.csv')
st2 = pd.read_csv('uni_rank_st.csv')
qt2 = pd.read_csv('uni_rank_qt.csv')
sr2 = pd.read_csv('uni_rank_sr.csv')
pr2 = pd.read_csv('uni_rank_pr.csv')
cm2 = pd.read_csv('uni_rank_cs.csv')
usn2 = pd.read_csv('uni_rank_usn.csv')
df_csr = pd.read_csv('ranks.csv')

pr2.USN2010 = pr2.USN2010.map(lambda x: int(x) if x.isnumeric() else np.nan)


In [ ]:
datasets = [
    [(row[2],row[1]) for row in st2.itertuples()],
    [(row[2],row[1]) for row in times.itertuples()],
    [(row[2],row[1]) for row in qt2.itertuples()],
    [(row[6],row[1]) for row in pr2.itertuples()],
    [(row[2],row[1]) for row in cm2.itertuples()],
    [(row[2],row[1]) for row in sr2.itertuples()],
    [(row[-1],row[2]) for row in srf2.itertuples()],

    [(row[0],idx+1) for idx,row in enumerate(t10_schools.itertuples())],
    [(row[2],row[1]) for row in df_csr.itertuples()],
    [(row[0],idx+1) for idx,row in enumerate(school_vals.itertuples())],
    [(row[0],idx+1) for idx,row in enumerate(school_vals.sort_values('pagerankA',0,False).itertuples())],
    [(row[0],idx+1) for idx,row in enumerate(school_vals.sort_values('pagerankC',0,False).itertuples())],
    [(row[0],idx+1) for idx,row in enumerate(school_vals.sort_values('Citations',0,False).itertuples())],
    [(row[0],idx+1) for idx,row in enumerate(school_vals.sort_values('papers',0,False).itertuples())],

    [(row[2],row[1]) for row in bp2.itertuples()],
    [(row[6],row[4]) for row in pr2.sort_values('NRC95',ascending=True).itertuples() ],
    [(row[6],row[3]) for row in pr2.sort_values('USN2010',ascending=True).itertuples() if np.isfinite(row[3]) ],
    [(row[2],row[1]) for row in usn2.itertuples()]
]
dataset_names = ['Shanghai','Times','QS','Prestige','CSMetrics',
                 'ScholarRank','ScholarRankFull','t10Sum','CSRankings','Mine','pageranka','pagerankc','citations','papers','BestPaper','NRC95',"USN10",'USN18']
n_datasets = len(datasets)


In [ ]:
school_vals

In [ ]:
corr_matrix = np.zeros((n_datasets,n_datasets))
count_matrix = np.zeros((n_datasets,n_datasets))
name_datasets = [ [v[0] for v in d] for d in datasets ]
all_names = sorted(list(set(sum(name_datasets,[]))))
all_vec = [sum([name in d for d in name_datasets])>=(len(datasets)-4) for name in all_names]
subset_names = [name for name,vec in zip(all_names,all_vec) if vec]
subset_names,len(subset_names)
import scipy.stats as stats
for i in range(n_datasets):
    inames = [u[0] for u in datasets[i]]
    for j in range(i,n_datasets):
        jnames = [u[0] for u in datasets[j]]

        #exist_1 = [((ni in subset_names) and (ni in jnames)) for ni in inames]
        #exist_2 = [((nj in subset_names) and (nj in inames))for nj in jnames]
        exist_1 = [((True) and (ni in jnames)) for ni in inames]
        exist_2 = [((True) and (nj in inames))for nj in jnames]
        
        d1 = np.array(datasets[i])[exist_1]
        d2 = np.array(datasets[j])[exist_2]
        v1 = d1[:,1].astype(np.float)
        v2 = np.array([d2[np.where(d2[:,0] == name)[0][0],1] for name in d1[:,0]]).astype(np.float)
        c = stats.kendalltau(v1,v2)[0]
        corr_matrix[i][j] = c
        corr_matrix[j][i] = c
        count_matrix[i][j] = len(v1)
        count_matrix[j][i] = len(v2)
        #print(v1,v2)

In [ ]:
np.set_printoptions(precision=2)
print(corr_matrix)
print('mean best')
for s,n in sorted([(s,n) for n,s in zip(dataset_names,corr_matrix.mean(1))],reverse=True):
    print('{:30s}\t{:.3f}'.format(n,s))
print('\n usnews best')
for s,n in sorted([(s,n) for n,s in zip(dataset_names,corr_matrix[-1])],reverse=True):
    print('{:30s}\t{:.3f}'.format(n,s))
print('\n names')

for n in dataset_names:
    print(n)

In [ ]:
plt.style.use('seaborn-whitegrid')

In [ ]:
from collections import Counter,defaultdict
def di():
    return defaultdict(float)
 
author_by_year = defaultdict(di)
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    for a in authors:
        author_by_year[a][year] += result_clf[years_per_conf*conf_idx[venue] + offset_years[year-min_year]]/n

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d
import matplotlib.gridspec as gridspec
gs = gridspec.GridSpec(4, 4)
plt.figure(figsize=(12,12))
example_names = ['Takeo Kanade','Michael I. Jordan','Leonidas J. Guibas','Daniela Rus','Jon M. Kleinberg', 'Judea Pearl']

plt.subplot(gs[2:4, 1:3])

for example_name in example_names:
    example_value = np.zeros(max_year+1-min_year)
    years = author_by_year[example_name]
    yrs = [_ for _ in years.keys() if _  > 0]
    start_year = min(yrs)
    end_year = max(yrs)
    span = end_year - start_year
    start_year,end_year,span
    for y,v in years.items():
        example_value[y-min_year] += v
            
    plt.plot(np.arange(min_year,max_year+1)[:-1],gaussian_filter1d(example_value[:-1], sigma=4.5),label=example_name,lw=4)
    #plt.plot(gaussian_filter1d(example_value[:span], sigma=2),label=example_name )

#plt.plot((val_by_year/v_count),label='average author')
plt.ylabel('annual value (4.5 sigma smoothing)')
#plt.xlabel('year since first publication')
plt.xlabel('year')
plt.title('venue scores')
plt.legend()
plt.grid(True)
plt.subplot(gs[:2, :2])
for example_name in example_names:
    example_value = np.zeros(max_year+1-min_year)
    for yr in range(min_year,max_year+1):
        if (example_name,yr) in paper_counts_year:
            example_value[yr-min_year] += paper_counts_year[example_name,yr]
    plt.plot(np.arange(min_year,max_year+1)[:-1],gaussian_filter1d(example_value[:-1], sigma=4.5),label=example_name,lw=4)
    #plt.plot(gaussian_filter1d(example_value[:span], sigma=2),label=example_name )

#plt.plot((val_by_year/v_count),label='average author')
plt.ylabel('annual value (4.5 sigma smoothing)')
#plt.xlabel('year since first publication')
plt.xlabel('year')
plt.title('paper count')
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.subplot(gs[:2, 2:])

for example_name in example_names:
    example_value = np.zeros(max_year+1-min_year)
    for yr in range(min_year,max_year+1):
        if (example_name,yr) in paper_counts_year:
            example_value[yr-min_year] += paper_counts_year[example_name,yr]
    plt.plot(np.arange(min_year,max_year+1)[:-1],gaussian_filter1d((example_value/paper_counts_year_counter)[:-1], sigma=4.5),label=example_name,lw=4)
    #plt.plot(gaussian_filter1d(example_value[:span], sigma=2),label=example_name )

#plt.plot((val_by_year/v_count),label='average author')
plt.ylabel('annual value (4.5 sigma smoothing)')
#plt.xlabel('year since first publication')
plt.xlabel('year')
plt.title('paper count (norm by annual paper totals)')
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.savefig('fame3.pdf',facecolor='w',edgecolor='w')

In [ ]:
ri_names = list(set([aliasdict.get(row[1],row[1]) for row in pd.read_csv('cmu_faculty.csv').itertuples() if row[2] == 'RI']))

In [ ]:
uni_names = np.array(['Artur Dubrawski','Artur W. Dubrawski'])
uni_names = list(uni_names)
cmu_scores = []
for name in uni_names:
    if name in name_idx:
        score = total_scores[name_idx[name]]
        cmu_scores.append(((score-tm)/ts,name))
for s,p in sorted(cmu_scores,reverse=True):
    print('{:30s}\t\t{:.3f}'.format(p,s))


In [ ]:
cmu_uni = pd.read_csv('cmu_faculty.csv')
cmu_uni = cmu_uni.fillna('Other')
cmu_uni = cmu_uni[cmu_uni.dept == 'RI']
#print(list(cmu_uni.name))
uni_names = list(cmu_uni.name)

In [ ]:
conf_counts = {}
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    if year < 2004:
        continue
    for a in authors:
        if a in uni_names:
            conf_counts[venue] = 1 + conf_counts.get(venue,0)

In [ ]:
for v,k in sorted([(v,k) for k,v in conf_counts.items() ],reverse=True):
    print('{:30s}\t{}'.format(k[:30],round(v)))

In [ ]:
len(conf_counts)

In [ ]:
prof_conf_counts = {}
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    if year < 2004:
        continue
    for a in authors:
        if a in uni_names:
            prof_conf_counts[venue] = [a] + prof_conf_counts.get(venue,[])

In [ ]:
confs_of_interest = ['CVPR','ICCV','ECCV']
for conf_of_interest in confs_of_interest:
    unique_set = {k: 0 for k in set(prof_conf_counts[conf_of_interest])}
    for k in prof_conf_counts[conf_of_interest]:
        unique_set[k] += 1


In [ ]:
for v,k in sorted([(v,k) for k,v in unique_set.items() ],reverse=True):
    print('{:30s}\t{}'.format(k[:30],round(v)))